In [7]:
# 📌 Cell 1: Setup and Load Pretrained Summarization Model

# Install required libraries if not already installed
#!pip install transformers torch --quiet

# Import necessary modules
from transformers import pipeline

# Load a powerful summarization pipeline using Facebook's BART large CNN model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

print("✅ Summarization model loaded and ready to use.")


Device set to use cuda:0


✅ Summarization model loaded and ready to use.


In [9]:
# 📌 Cell 2: Input Lengthy Article and Generate Summary

# Example lengthy text (replace with any article)
article_text = """
In the age of information overload, summarization tools are increasingly vital for digesting massive volumes of text quickly and efficiently. 
Modern natural language processing (NLP) techniques leverage large pretrained models to extract and condense key information from documents, 
news articles, research papers, and more. These models are trained on diverse datasets and can understand the nuances of human language, 
producing summaries that are coherent and contextually relevant. One of the leading models in this domain is Facebook's BART, which has 
achieved state-of-the-art results in text summarization tasks. By integrating such models into easy-to-use tools, both individuals and 
organizations can save time, enhance productivity, and make informed decisions based on concise insights.
"""

# Generate summary with practical high max_length
summary = summarizer(
    article_text,
    min_length=10,    # minimum summary length
    max_length=1024,  # high value to avoid cutting off too soon
    do_sample=False   # deterministic output
)

print("🔹 Original Text:\n", article_text)
print("\n🔹 Generated Summary:\n", summary[0]['summary_text'])


Your max_length is set to 1024, but your input_length is only 163. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=81)


🔹 Original Text:
 
In the age of information overload, summarization tools are increasingly vital for digesting massive volumes of text quickly and efficiently. 
Modern natural language processing (NLP) techniques leverage large pretrained models to extract and condense key information from documents, 
news articles, research papers, and more. These models are trained on diverse datasets and can understand the nuances of human language, 
producing summaries that are coherent and contextually relevant. One of the leading models in this domain is Facebook's BART, which has 
achieved state-of-the-art results in text summarization tasks. By integrating such models into easy-to-use tools, both individuals and 
organizations can save time, enhance productivity, and make informed decisions based on concise insights.


🔹 Generated Summary:
 In the age of information overload, summarization tools are increasingly vital for digesting massive volumes of text quickly and efficiently. One of the le

In [10]:
# 📌 Cell 3: Load Article from File or Manual Input

import os

# Option 1: Load from a .txt file
file_path = "article.txt"  # Replace with your file name

if os.path.exists(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        article_text = file.read()
    print(f"✅ Loaded article from '{file_path}'")
else:
    print(f"⚠️ File '{file_path}' not found. Using manual input instead.")
    # Option 2: Manual input fallback
    article_text = input("Please paste your article text here:\n")

print("\n🔹 Article Preview (first 500 characters):")
print(article_text[:500] + ("..." if len(article_text) > 500 else ""))


✅ Loaded article from 'article.txt'

🔹 Article Preview (first 500 characters):
One of India’s greatest assets is youth. Jayan Jose Thomas calls for changes in established thinking to harness its potential.

India, like many South Asian countries, has a young population. This is at once an opportunity for future economic growth, but it presents policymakers with the challenge of creating employment. According to the World Bank, the projected increase in South Asia’s working-age population during 2020-2050 is some 254.m accounting for 30.6% of the increase worldwide. India a...


In [12]:
# 📌 Cell 4: Robust Summarization with CPU and Chunking

from math import ceil

# Force summarizer to run on CPU to avoid CUDA errors
summarizer_cpu = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=-1  # -1 means CPU
)

# BART max input length: ~1024 tokens (~1024 words for English)
max_chunk_words = 500  # safe chunk size

# Split text into chunks
words = article_text.split()
num_chunks = ceil(len(words) / max_chunk_words)
chunks = [
    " ".join(words[i*max_chunk_words : (i+1)*max_chunk_words])
    for i in range(num_chunks)
]

print(f"✅ Splitting article into {num_chunks} chunk(s) for safe summarization.\n")

# Summarize each chunk, then join results
summaries = []
for i, chunk in enumerate(chunks):
    print(f"🔹 Summarizing chunk {i+1}/{num_chunks}...")
    summary = summarizer_cpu(
        chunk,
        min_length=10,
        max_length=200,   # smaller chunk summary
        do_sample=False
    )
    summaries.append(summary[0]['summary_text'])

# Combine summaries
final_summary = "\n\n".join(summaries)

print("\n🔹 Final Combined Summary:\n")
print(final_summary)


Device set to use cpu


✅ Splitting article into 3 chunk(s) for safe summarization.

🔹 Summarizing chunk 1/3...
🔹 Summarizing chunk 2/3...
🔹 Summarizing chunk 3/3...

🔹 Final Combined Summary:

Jayan Jose Thomas calls for changes in established thinking to harness its potential. India, like many South Asian countries, has a young population. Increasing enrolment in education and a growing exodus of young people from agriculture are swelling the jobless ranks.

Of India’s total workforce of 471.0 million, only 12.3% are regular workers who receive some form of social security and are covered, at least partially, by laws that protect labour rights. The rest are mostly casual workers or petty producers surviving under various degrees of informality.

India’s working classes deserve real improvements in living and working conditions. This can happen only with a massive expansion in government spending. Proposals might include the setting up of industries linked to food processing or affordable housing in rural ar

In [16]:
# 📌 Cell 5: Save Final Summary to File

output_file = "summary.txt"

try:
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(final_summary)
    print(f"✅ Summary saved successfully to '{output_file}'")
except Exception as e:
    print(f"⚠️ Failed to save summary: {e}")

✅ Summary saved successfully to 'summary.txt'
